# 爱科研-人工智能科研实训项目
## 支持向量机（Support Vector Machine, SVM）

## 项目：计算机视觉 - 车辆检测

在这个notebook文件中，有些模板代码已经提供给你，但你还需要实现更多的功能来完成这个项目。除非有明确要求，你无须修改任何已给出的代码。以**'(练习)'**开始的标题表示接下来的代码部分中有你需要实现的功能。这些部分都配有详细的指导，需要实现的部分也会在注释中以'TODO'标出。请仔细阅读所有的提示。

除了实现代码外，你还**需要**回答一些与项目及代码相关的问题。每个需要回答的问题都会以 **'问题 X'** 标记。请仔细阅读每个问题，并且在问题后的 **'回答'** 部分写出完整的答案。我们将根据 你对问题的回答 和 撰写代码实现的功能 来对你提交的项目进行评分。

>**提示：**Code 和 Markdown 区域可通过 **Shift + Enter** 快捷键运行。此外，Markdown可以通过双击进入编辑模式。

### 让我们开始吧
在这个notebook中，你将在交通标志识别之后，继续开发可以作为自动驾驶汽车视觉一部分的算法。本项目采用图像HOG特征结合SVM分类器的方法对摄像机画面 中的车辆进行探测（Detection）。在本次爱科研科研实训毕业项目中，你也将尝试使用神经网络对画面中的车辆进行探测。

![Sample Output](images/preview.png)

### 项目内容

* 对有标记的图像提取[方向梯度直方图](https://blog.csdn.net/zouxy09/article/details/7929348)（HOG）特征，并利用该特征训练支持向量机（SVM）分类器
* 使用滑动窗口（Sliding-Window）配合SVM分类器寻找摄像机画面中的车辆
* 利用上述算法对视频 `test_video.mp4` 中的车辆进行实时探测，并使用热图检测异常值
* 对探测出的车辆的边界框（Bounding Box）进行估计
---

我们将这个notebook分为不同的步骤，你可以使用下面的链接来浏览此notebook。

* [Step 0](#step0): SVM自我评估
* [Step 1](#step1): 数据预览
* [Step 2](#step2): HOG特征提取
* [Step 3](#step3): 滑动窗口
* [Step 4](#step4): 处理视频

在该项目中包含了如下的问题：

* [问题 1](#question1): 简要概括硬间隔SVM优化目标。
* [问题 2](#question2): 采用什么方式让SVM可以处理线性不可分问题？
* [问题 3](#question3): 引入对偶问题的优势在哪里？
* [问题 4](#question4): 默写凸优化问题的KKT条件。
* [问题 5](#question5): 说明 `SVC` 与 `LinearSVC` 之间的不同点。

---
<a id='step0'></a>
## 步骤 0: SVM自我评估

<a id='question1'></a>  

### __问题 1:__ 

简要概括硬间隔SVM优化目标：
> minimize $\frac{1}{2}\|{\bf w}\|^2$ subject to $y_i({\bf w}^T{\bf x}_i+b)\geq1$

的推导过程。

__回答:__ 

<a id='question2'></a>  

### __问题 2:__ 

硬间隔SVM无法处理线性不可分问题。那么，采用什么方式让SVM可以处理线性不可分问题？

__回答:__ 

<a id='question3'></a>  

### __问题 3:__ 

求解SVM时是否一定需要求解其对偶问题？什么时候需要引入对偶问题？引入对偶问题的优势在哪里？

__回答:__ 

<a id='question4'></a>  

### __问题 4:__ 

KKT条件为凸优化问题强对偶成立的充要条件。写出该凸优化问题：

> minimize $f_0({\bf x})$ suject to 

> $f_i({\bf x})\leq 0$ for $i=0,1,2,\cdots,m$ 

> $h_i({\bf x})= 0$ for $i=0,1,2,\cdots,p$ 

的KKT条件。（引入Lagrangian函数）

__回答:__ 

---
<a id='step1'></a>
## 步骤 1: 数据预览

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from glob import glob
from tqdm import tqdm
%matplotlib inline

In [ ]:
car_images = glob('./data/vehicles/**/*.png')
noncar_images = glob('./data/non-vehicles/**/*.png')

print('Number of car images: ',len(car_images))
print('Number of non-car images: ',len(noncar_images))

In [ ]:
### 随机展示部分图像
fig, axs = plt.subplots(8,8, figsize=(16, 16))
fig.subplots_adjust(hspace = .2, wspace=.001)
axs = axs.ravel()

for i in np.arange(32):
    img = cv2.imread(car_images[np.random.randint(0,len(car_images))])
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    axs[i].axis('off')
    axs[i].set_title('car', fontsize=10)
    axs[i].imshow(img)
for i in np.arange(32,64):
    img = cv2.imread(noncar_images[np.random.randint(0,len(noncar_images))])
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    axs[i].axis('off')
    axs[i].set_title('non-car', fontsize=10)
    axs[i].imshow(img)

---

<a id='step2'></a>
## 步骤 2: HOG特征提取

[方向梯度直方图](https://blog.csdn.net/zouxy09/article/details/7929348)（Histogram of Oriented Gradient, HOG）特征是一种在计算机视觉和图像处理中用来进行物体检测的特征描述子。它通过计算和统计图像局部区域的梯度方向直方图来构成特征。Hog特征结合SVM分类器已经被广泛应用于图像识别中，尤其在行人检测中获得了极大的成功。需要提醒的是，HOG+SVM进行行人检测的方法是法国研究人员Dalal在2005的CVPR上提出的，而如今虽然有很多行人检测算法不断提出，但基本都是以HOG+SVM的思路为主。

![HOG Output](images/hog.png)

---
### HOG特征示例

本项目使用 `scikit-image` 中的 [hog](http://scikit-image.org/docs/dev/api/skimage.feature.html#skimage.feature.hog) 函数进行HOG特征提取。

In [ ]:
from skimage.feature import hog

def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=False, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=False, 
                       visualise=vis, feature_vector=feature_vec)
        return features

In [ ]:
### HOG特征示例
car_img = mpimg.imread(car_images[5])
_, car_dst = get_hog_features(car_img[:,:,2], 9, 8, 8, vis=True, feature_vec=True)
noncar_img = mpimg.imread(noncar_images[5])
_, noncar_dst = get_hog_features(noncar_img[:,:,2], 9, 8, 8, vis=True, feature_vec=True)

f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(8,8))
f.subplots_adjust(hspace = .4, wspace=.2)
ax1.imshow(car_img)
ax1.set_title('Car Image', fontsize=16)
ax2.imshow(car_dst, cmap='gray')
ax2.set_title('Car HOG', fontsize=16)
ax3.imshow(noncar_img)
ax3.set_title('Non-Car Image', fontsize=16)
ax4.imshow(noncar_dst, cmap='gray')
ax4.set_title('Non-Car HOG', fontsize=16)
plt.show()

### 提取图像的HOG特征

In [ ]:
### 添加空间特征
def bin_spatial(img ,size=(32,32)):
    features=cv2.resize(img, size)
    return features.ravel()

### 添加颜色直方图特征
def color_hist(img, hist_bins=32, color_range=(0,256)):
    ch1=np.histogram(img[:,:,0], hist_bins, range=color_range)
    ch2=np.histogram(img[:,:,1], hist_bins, range=color_range)
    ch3=np.histogram(img[:,:,2], hist_bins, range=color_range)
    hist_features=np.hstack((ch1[0], ch2[0], ch3[0]))
    return hist_features

def single_img_features(img, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):    
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)      
    #3) Compute spatial features if flag is set
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))      
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, hist_bins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    
    #9) Return concatenated array of features
    return np.concatenate(img_features)

def extract_features(imgs, cspace='RGB', orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,spatial_size=(16,16),hist_bins=32,spatial_feet=True, hist_feet=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = cv2.imread(file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img_feature = single_img_features(image, color_space=cspace, spatial_size=spatial_size,
                        hist_bins=hist_bins, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, hog_channel=hog_channel,
                        spatial_feat=spatial_feet, hist_feat=hist_feet, hog_feat=True)
        features.append(img_feature)
    
    # Return list of feature vectors
    return features

In [ ]:
#TODO: explore the parameters below 

### HOG特征可选参数
colorspace = None # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9
pix_per_cell = 8
cell_per_block = 2
hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"

### 额外特征参数
spatial_size=(16,16)
hist_bins=32

import time

start = time.time()
car_features = extract_features(car_images, cspace=colorspace, orient=orient, pix_per_cell=pix_per_cell, 
                                cell_per_block=cell_per_block, hog_channel=hog_channel,spatial_size=spatial_size,
                                hist_bins=hist_bins, spatial_feet=True, hist_feet=True)
noncar_features = extract_features(noncar_images, cspace=colorspace, orient=orient, pix_per_cell=pix_per_cell, 
                                   cell_per_block=cell_per_block, hog_channel=hog_channel,spatial_size=spatial_size, 
                                   hist_bins=hist_bins, spatial_feet=True, hist_feet=True)
end = time.time()

print('Using:',orient,'orientations',pix_per_cell,'pixels per cell and', cell_per_block,'cells per block')
print(round(end-start, 2), 'Seconds to extract HOG feature')
print('HOG Feature vector length:', len(car_features[0]))

### 特征长度示例

In [ ]:
example = cv2.imread(car_images[0])

hog_example = single_img_features(example, color_space=colorspace, spatial_size=spatial_size,
                        hist_bins=hist_bins, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, hog_channel=hog_channel,
                        spatial_feat=False, hist_feat=False, hog_feat=True)
spatial_example = single_img_features(example, color_space=colorspace, spatial_size=spatial_size,
                        hist_bins=hist_bins, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, hog_channel=hog_channel,
                        spatial_feat=True, hist_feat=False, hog_feat=False)
hist_example = single_img_features(example, color_space=colorspace, spatial_size=spatial_size,
                        hist_bins=hist_bins, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, hog_channel=hog_channel,
                        spatial_feat=False, hist_feat=True, hog_feat=False)
print('HOG:',len(hog_example), ' Spatial:',len(spatial_example), ' Color:',len(hist_example))

plt.figure(figsize=(8,1))
plt.barh(0.0, len(hog_example), 0.2, color='r',left=0)
plt.barh(0.0, len(spatial_example),0.2, color='g',left=len(hog_example))
plt.barh(0.0, len(hist_example), 0.2, color='b', left=len(hog_example)+len(spatial_example))
plt.xlim(0,10000)
plt.legend(['hog features', 'spatial features', 'color histogram features'], loc='upper right')
plt.title('The length of three kinds of features')
plt.show()

### 合并特征并划分数据集

使用 `vstack` 与 `hstack` 将不同图片的HOG特征向量合并在一起。`numpy` 中数组合并相关的操作示意图如下：

![Stack Output](images/stack.png)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Create an array stack of feature vectors
X = np.vstack((car_features, noncar_features)).astype(np.float64)

# Normalize
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(noncar_features))))

# TODO: Split up data into randomized training and test sets



print('The size of training set is', len(X_train))
print('The size of validation set is', len(X_val))

---
<a id='step2'></a>
## 步骤 2: 训练SVM分类器

<a id='question5'></a>  

### __问题 5:__ 

本项目中从 `sklearn.svm` 中导入了 `SVC` 与 `LinearSVC` 两种分类器，查阅资料说明它们之间的不同点。

> sklearn文档链接：[SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)，[LinearSVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html)

__回答:__ 

---

### 选择分类器

分别使用 `SVC` 与 `LinearSVC` 分类器对数据进行训练，完成下表，并依此选择最合适的分类器。

**Exploration**

| Classifier | Accuracy | Train Time |
| :--------: | -------: | ---------: |
| SVC      |       |         |
| Linear SVC |       |         |

In [ ]:
from sklearn.svm import SVC, LinearSVC

start = time.time()
svc = SVC(random_state=233)
svc.fit(X_train, y_train)
end = time.time()
print(round(end-start, 2), 'Seconds to train the SVC Classifier')
print('Test Accuracy of SVC = ', round(svc.score(X_val, y_val), 4))

start = time.time()
lsvc = LinearSVC(random_state=233)
lsvc.fit(X_train, y_train)
end = time.time()
print(round(end-start, 2), 'Seconds to train the LinearSVC Classifier')
print('Test Accuracy of LinearSVC = ', round(lsvc.score(X_val, y_val), 4))

### 使用网格搜索确认最佳超参数

In [ ]:
#TODO: find the best model and penalty parameter C_best

model = None

print('The best penalty parameter C is ', round(C_best,3))
print('Test Accuracy of LinearSVC = ', round(model.score(X_val, y_val), 4))

# Check the prediction time for a series of samples
n_predict = 10
print('Predictions for 10 samples: ', model.predict(X_val[0:n_predict]))
print('True labels for 10 samples: ', y_val[0:n_predict])

---
<a id='step3'></a>
## 步骤 3: 滑动窗口

滑动窗口（Sliding Windows，简称滑窗）法是进行目标检测的主流方法。对于某输入图像，由于其对象尺度形状等因素的不确定性，导致直接套用预训练好的模型进行识别效率低下。通过设计滑窗来遍历图像，将每个窗口对应的局部图像进行检测，能有效克服尺度、位置、形变等带来的输入异构问题，提升检测效果。下图展示了某种大小的滑窗在待检测图像上滑动的过程：

![SW Output](images/concept_sliding_windows.gif)

---
### 定义滑动窗口

In [ ]:
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy

def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list

In [ ]:
test_imgs = glob('./data/test_images/*.jpg')
image = mpimg.imread('./data/test_images/test4.jpg')
windows = slide_window(image, x_start_stop=[None, None], y_start_stop=[350, None], 
                    xy_window=(128, 128), xy_overlap=(0.5, 0.5))             
window_img = draw_boxes(image, windows, color=(0, 0, 255), thick=6)                    
plt.imshow(window_img)
plt.show()

### 滑动窗口搜寻

In [ ]:
def search_windows(img, windows, clf, scaler, color_space='RGB', 
                    spatial_size=(16, 16), hist_bins=16, 
                    hist_range=(0, 256), orient=9, 
                    pix_per_cell=8, cell_per_block=2, 
                    hog_channel=0, spatial_feat=True, 
                    hist_feat=True, hog_feat=True):
    on_windows = []
    for window in windows:
        test_img = img[window[0][1]:window[1][1],window[0][0]:window[1][0],:]
        test_img = cv2.resize(test_img, (64,64))
        feature = single_img_features(test_img, color_space=color_space, spatial_size=spatial_size,
                                      hist_bins=hist_bins,orient=orient, pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                                      hog_channel=hog_channel, spatial_feat=spatial_feat, hist_feat=hist_feat, hog_feat=hog_feat)
        
        feature = scaler.transform(feature.reshape(1,-1))
        predict = clf.predict(feature)
        if predict == 1:
            on_windows.append(window)
    return on_windows

In [ ]:
### 绘图函数
def add_heat(image, hot_windows):
    mask=np.zeros_like(image[:,:,0])
    for window in hot_windows:
        mask[window[0][1]:window[1][1],window[0][0]:window[1][0]] += 1
    return mask

def apply_threshold(heat, threshold):
    heat[heat<threshold] = 0
    return heat

def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img

In [ ]:
xy_windows = [(96,96),(128,128)]
image_windows = []

for name in test_imgs:
    image = cv2.imread(name)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    total_windows = []
    for xy_window in xy_windows:
        windows = slide_window(image, x_start_stop=[600, None], y_start_stop=[350, 660], 
                    xy_window=xy_window, xy_overlap=(0.8, 0.8))
        total_windows.extend(windows)
    
    hot_windows = search_windows(image, total_windows, model, scaler, color_space=colorspace, 
                                 spatial_size=(16, 16), hist_bins=hist_bins, hist_range=(0, 256), 
                                 orient=orient, pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                                 hog_channel=hog_channel, spatial_feat=True, hist_feat=True, hog_feat=True)
    image_windows.append(hot_windows)
    draw_img = draw_boxes(image, hot_windows)
    heatmap = add_heat(image, hot_windows)
    heatmap = apply_threshold(heatmap, 2)
    plt.figure(figsize=(10,8))
    plt.subplot(1,2,1)
    plt.imshow(draw_img)
    plt.subplot(1,2,2)
    plt.imshow(heatmap,cmap='hot')
    plt.show()

In [ ]:
### 根据滑动窗口识别结果画出Bounding Box
from scipy.ndimage.measurements import label

plt.figure(figsize=(16,7))
for i,name in enumerate(test_imgs):
    img=mpimg.imread(name)
    window_img = draw_boxes(img, image_windows[i], color=(0, 0, 255), thick=6)                    
    heatmap=add_heat(img, image_windows[i])
    heatmap=apply_threshold(heatmap, 2)
    labels=label(heatmap)
    draw_img = draw_labeled_bboxes(np.copy(img), labels)
    plt.subplot(2,3,i+1)
    plt.title('example %d'%(i+1))
    plt.imshow(draw_img)
plt.show()

---
<a id='step4'></a>
## 步骤 4: 处理视频

In [ ]:
def preprocess(image):
    xy_windows=[(96,96),(128,128)]
    total_windows=[]
    for xy_window in xy_windows:
        windows = slide_window(image, x_start_stop=[600, None], y_start_stop=[350,660], 
                    xy_window=xy_window, xy_overlap=(0.8, 0.8))
        total_windows.extend(windows)    

    hot_windows = search_windows(image, total_windows, model, scaler, color_space=colorspace, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=True, 
                        hist_feat=True, hog_feat=True)                       
    window_img = draw_boxes(image, hot_windows, color=(0, 0, 255), thick=6)                    
    heatmap=add_heat(image, hot_windows)
    heatmap=apply_threshold(heatmap, 2)
    labels=label(heatmap)
    draw_img = draw_labeled_bboxes(np.copy(image), labels)
    return draw_img

In [ ]:
from moviepy.editor import VideoFileClip

output = 'test_video_output.mp4'
clip1 = VideoFileClip("./data/test_video.mp4")
#clip1 = VideoFileClip("project_video.mp4").subclip(20,28)

out_clip = clip1.fl_image(preprocess) #NOTE: this function expects color images!!
%time out_clip.write_videofile(output, audio=False)

In [ ]:
# show the video
from IPython.display import HTML

HTML("""<video width="960" height="540" controls><source src="{0}"></video>""".format(output))